In [22]:
import pandas as pd
import numpy as np

In [28]:
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation, cosine
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.metrics import pairwise_distances
from sklearn.metrics import mean_squared_error
from math import sqrt
import sys, os
from contextlib import contextmanager

In [114]:
book_tags = pd.read_csv("C:\\Users\\abila\\Desktop\\dataset\\book_tags.csv")
books = pd.read_csv("C:\\Users\\abila\\Desktop\\dataset\\books.csv")
ratings = pd.read_csv("C:\\Users\\abila\\Desktop\\dataset\\ratings.csv")
tags = pd.read_csv("C:\\Users\\abila\\Desktop\\dataset\\tags.csv")
to_read = pd.read_csv("C:\\Users\\abila\\Desktop\\dataset\\to_read.csv")

In [115]:
book_tags.drop_duplicates()
ratings.drop_duplicates()
counts1 = ratings["user_id"].value_counts()
counts2 = ratings["book_id"].value_counts()
ratings = ratings[ratings["user_id"].isin(counts1[counts1 >= 100].index)]
ratings = ratings[ratings["book_id"].isin(counts2[counts2 >= 100].index)]

In [126]:
n_books = ratings['book_id'].nunique()
n_users = ratings['user_id'].nunique()
UserRatingMatrix = np.zeros((n_books,n_users))
book_l = ratings["book_id"].drop_duplicates()
user_l = ratings["user_id"].drop_duplicates()

In [127]:
Ratings_df = pd.DataFrame(UserRatingMatrix, index = book_l, columns = user_l)

In [138]:
for index, rows in ratings.iterrows():
    Ratings_df.loc[rows['book_id']][rows['user_id']] = rows['rating']

In [140]:
Ratings_df = Ratings_df.T

In [141]:
Ratings_df

book_id,1,2,3,4,5,6,7,8,9,10,...,9960,9963,9964,9966,9976,9977,9981,9990,9994,9995
user_id,,,,,,,,,,,,,,,,,,,,,
314,5.0,0.0,3.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
439,3.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
588,5.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1169,4.0,3.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1185,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [188]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute') 
model_knn.fit(Ratings_df)
global model
model = model_knn

In [189]:
def findksimilarusers(user_id, ratings, model, metric = 'cosine', k=4):
    similarities=[]
    indices=[]
    distances, indices = model.kneighbors(ratings.iloc[user_id, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()
    print ('{0} most similar users for User {1}:\n'.format(k,user_id))
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == user_id:
            continue;

        else:
            print ('{0}: User {1}, with similarity of {2}'.format(i, indices.flatten()[i], similarities.flatten()[i]))
            
    return similarities,indices

In [185]:
def predict_userbased(user_id, item_id, ratings, model, metric = 'cosine', k=4):
    prediction=0
    similarities, indices=findksimilarusers(user_id, ratings,model,metric, k) #similar users based on cosine similarity
    mean_rating = ratings.loc[user_id,:].mean() #to adjust for zero based indexing
    sum_wt = np.sum(similarities)-1
    product=1
    wtd_sum = 0 
    
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i] == user_id:
            continue;
        else: 
            ratings_diff = ratings.iloc[indices.flatten()[i],item_id]-np.mean(ratings.iloc[indices.flatten()[i],:])
            product = ratings_diff * (similarities[i])
            wtd_sum = wtd_sum + product
    
    prediction = int(round(mean_rating + (wtd_sum/sum_wt)))
    print ('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))

    return prediction

In [190]:
predict_userbased(314,2,Ratings_df, model = model)

4 most similar users for User 314:

0: User 314, with similarity of 1.0
1: User 92, with similarity of 0.24280523787069974
2: User 162, with similarity of 0.24076195971025682
3: User 165, with similarity of 0.23670790935055697
4: User 164, with similarity of 0.23670790935055697

Predicted rating for user 314 -> item 2: 5


5

In [191]:
import pickle
pickle.dump(model, open("C:\\Users\\abila\\Desktop\\dataset\\br_model.pkl", 'wb'))

In [192]:
loaded_model = pickle.load(open("C:\\Users\\abila\\Desktop\\dataset\\br_model.pkl", 'rb'))
type(loaded_model)

sklearn.neighbors._unsupervised.NearestNeighbors